# 03 — Risk Measures (DV01 / Duration / Convexity)

## Objective
Compute interest-rate risk measures using finite differences and consistency checks:
- DV01 per bond and portfolio
- Modified duration from DV01
- Convexity (2nd-order sensitivity)

## Definitions
With yield shock $\Delta y$:
$$
DV01 \approx P(y+\Delta y) - P(y)
$$
(Here DV01 is per 1bp shock, so $\Delta y = 0.0001$.)

Modified duration estimate:
$$
D_{mod} \approx -\frac{DV01}{P \cdot 0.0001}.
$$

Convexity (symmetric FD):
$$
Conv \approx \frac{P(y+\Delta y)+P(y-\Delta y)-2P(y)}{P(y)\Delta y^2}.
$$

In [1]:
import pandas as pd

from fixed_income_engine.curves import bootstrap_curve_from_bills_notes
from fixed_income_engine.portfolio import make_sample_portfolio
from fixed_income_engine.bonds import settlement_date
from fixed_income_engine.risk import (
    compute_portfolio_dv01,
    compute_portfolio_convexity,
    duration_from_dv01,
)

val_date = pd.Timestamp("2026-02-13")
settle = pd.Timestamp("2026-02-16")

market = pd.DataFrame([
    {"type": "bill", "maturity": pd.Timestamp("2026-02-20"), "quote": 0.0525, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-03-13"), "quote": 0.0520, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-05-15"), "quote": 0.0515, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-08-14"), "quote": 0.0505, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2027-02-12"), "quote": 0.0485, "day_count": "ACT/360"},
    {"type": "note", "maturity": pd.Timestamp("2028-02-15"), "quote": 0.0450, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2031-02-15"), "quote": 0.0430, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2036-02-15"), "quote": 0.0425, "coupon_freq": 2, "day_count": "30/360"},
]).sort_values("maturity").reset_index(drop=True)

curve = bootstrap_curve_from_bills_notes(market, val_date)

portfolio_df = make_sample_portfolio(n=20, val_date=val_date, seed=7)
portfolio_df.head()

,bond_id,maturity,coupon_rate,freq,day_count,face
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0
1,BOND_001,2033-02-15,0.036706,2,30/360,100.0
2,BOND_002,2033-02-15,0.035292,2,30/360,100.0
3,BOND_003,2035-02-15,0.046705,2,30/360,100.0
4,BOND_004,2032-02-15,0.050273,2,30/360,100.0


In [2]:
dv01_df = compute_portfolio_dv01(curve, portfolio_df, val_date, settle)
dv01_df.head()

,bond_id,dirty_base,dirty_up1bp,dv01
0,BOND_000,94.504154,94.425087,-0.079067
1,BOND_001,94.369914,94.311254,-0.058660
2,BOND_002,93.539742,93.481378,-0.058364
3,BOND_003,100.923368,100.847944,-0.075424
4,BOND_004,101.723256,101.669695,-0.053561


In [3]:
dur_df = duration_from_dv01(dv01_df)
dur_df.head()

,bond_id,mod_duration
0,BOND_000,8.366467
1,BOND_001,6.215978
2,BOND_002,6.239485
3,BOND_003,7.473386
4,BOND_004,5.265350


In [4]:
conv_df = compute_portfolio_convexity(curve, portfolio_df, val_date, settle)
conv_df.head()

,bond_id,convexity
0,BOND_000,78.385251
1,BOND_001,41.673294
2,BOND_002,41.894041
3,BOND_003,62.760942
4,BOND_004,30.077598


## What matters about these results
- DV01 is mostly negative: prices fall when rates rise (normal duration behavior).
- Convexity is mostly positive: curvature reduces linear approximation error for larger shocks.
- Duration-from-DV01 is a **consistency check**: if your DV01 is computed correctly, duration should be stable and sensible.
